In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install evaluate

In [3]:
from huggingface_hub import login, notebook_login
from transformers import AutoTokenizer
import transformers
import torch
import evaluate

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `AutoGen` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `AutoGen`


In [5]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [6]:
# DistilBERT base model (uncased)
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def QAinference(model, tokenizer, question, context, device='cpu'):
  inputs = tokenizer(question, context, return_tensors="pt")
  inputs = {k: v.to(device) for k, v in inputs.items()}
  with torch.no_grad():
    outputs = model(**inputs)
  #Get the highest probability from the model output for the start and end positions:
  absnswer_start_index = outputs.start_logits.argmax()
  answer_end_index = outputs.end_logits.argmax()
  #Decode the predicted tokens to get the answer:
  predict_answer_tokens = inputs['input_ids'][0, answer_start_index : answer_end_index + 1]
  answers=tokenizer.decode(predict_answer_tokens)
  print(answers)
  return answers

In [8]:
device='cpu'

question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."
inputs = tokenizer(question, context, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}
with torch.no_grad():
  outputs = model(**inputs)
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ 0.0776, -0.1055, -0.2345,  0.0619,  0.1736, -0.1120,  0.2506, -0.0655,
         -0.1027, -0.0181,  0.3123, -0.3275, -0.2210,  0.0036,  0.1404,  0.0192,
         -0.0372,  0.1300,  0.1791,  0.2216, -0.2224,  0.0805,  0.0324,  0.2464,
         -0.0168, -0.2584,  0.1131,  0.2385,  0.1789, -0.0180]]), end_logits=tensor([[ 0.0615,  0.0148, -0.4283, -0.2630, -0.4699, -0.0773,  0.2058, -0.0460,
          0.0844, -0.1806,  0.3480, -0.2386, -0.2236, -0.5032, -0.2661, -0.0292,
         -0.3307, -0.1201,  0.0576,  0.0241,  0.0124, -0.2825,  0.0072, -0.2330,
          0.0324,  0.0114, -0.1206, -0.2817, -0.3246, -0.1807]]), hidden_states=None, attentions=None)

In [9]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [10]:
#Decode the predicted tokens to get the answer
predict_answer_tokens = inputs['input_ids'][0, answer_start_index : answer_end_index + 1]
answers=tokenizer.decode(predict_answer_tokens)
print(answers)

bloom


In [11]:
device='cpu'
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."
answers=QAinference(model, tokenizer, question, context)

bloom


In [12]:
metric = evaluate.load("squad")

In [13]:
from transformers import DefaultDataCollator
from torch.utils.data import DataLoader

In [ ]:
!pip install datasets

In [15]:
from datasets import load_dataset

In [16]:
raw_datasets = load_dataset("squad")

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [17]:
max_length = 384
stride = 128
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]] #100 questions
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")#100, if no overflow, then sample_map=0-99
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx]) #get example id strings

        sequence_ids = inputs.sequence_ids(i) #[None, 0... None, 1... 1]
        offset = inputs["offset_mapping"][i] #100 size array of tuple (0, 4)
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ] #put None in sequence_id==1, i.e., put questions to None

    inputs["example_id"] = example_ids #string list
    return inputs

In [18]:
valkeyname="validation"
small_eval_set = raw_datasets[valkeyname].select(range(100))

In [19]:
validation_dataset = small_eval_set.map(
            preprocess_validation_examples, #preprocess_function, #preprocess_validation_examples,
            batched=True,
            remove_columns=raw_datasets[valkeyname].column_names,
        )

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [20]:
print(len(raw_datasets[valkeyname]))
print(len(validation_dataset))

10570
100


In [21]:
eval_set_for_model = validation_dataset.remove_columns(["example_id", "offset_mapping"])

In [22]:
print(validation_dataset.features.keys())#['input_ids', 'attention_mask', 'offset_mapping', 'example_id']
print(eval_set_for_model.features.keys())#['input_ids', 'attention_mask']

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'example_id'])
dict_keys(['input_ids', 'attention_mask'])


In [23]:
eval_set_for_model.set_format("torch")

In [24]:
batch_size =32
data_collator = DefaultDataCollator()
eval_dataloader = DataLoader(
  eval_set_for_model, batch_size=batch_size, collate_fn=data_collator
    )

In [25]:
num_val_steps = len(eval_dataloader)
start_logits = []
end_logits = []

In [26]:
from tqdm import tqdm

In [27]:
valprogress_bar = tqdm(range(num_val_steps))
for batch in eval_dataloader:
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
  start_logits.append(outputs.start_logits.cpu().numpy())
  end_logits.append(outputs.end_logits.cpu().numpy())
  valprogress_bar.update(1)

100%|██████████| 4/4 [01:13<00:00, 15.40s/it]

In [28]:
def compute_metrics(start_logits, end_logits, features, examples):
    n_best = 20
    max_answer_length = 30

    #features is after tokenization, examples are original dataset
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [29]:
import numpy as np
import collections

In [30]:
# Evalation with 100 smapels
start_logits = np.concatenate(start_logits)
end_logits = np.concatenate(end_logits)
dataset_len=len(validation_dataset)
start_logits = start_logits[: dataset_len]
end_logits = end_logits[: dataset_len]
metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets[valkeyname]
    )
print(metrics)


100%|██████████| 10570/10570 [00:01<00:00, 10279.80it/s]


{'exact_match': 0.02838221381267739, 'f1': 0.02931890036874938}


In [31]:
print(metrics)

{'exact_match': 0.02838221381267739, 'f1': 0.02931890036874938}
